## 基本資料清理

空資料

資料撈錯,cbody網頁本身資料有誤 
    刪除字數<12(根據觀察)

*混雜code(css,javascript)
->留到斷詞完處理

重複資料處理(全留,隨機留,全砍,*留最少* imbalance data weight)

    同類別的重複資料(內文重複,url重複)-同類別下統一刪除內文重複


其他：

繁體轉簡體(方便jeiba切割)
openCC 繁體轉簡體用詞差異
(未做)


資料格式:

標籤(數字) title content(未切)

*這篇先不做斷句

In [1]:
import re
import pandas as pd

import numpy as np
import os
from os import listdir

# word cut system
import jieba

# 繁體簡體轉換
from hanziconv import HanziConv

In [4]:
os.chdir('../')
os.getcwd()

'/project/jt071-group23'

In [5]:
news_path = '/project/jt071-group23/news'
files = listdir(news_path)

In [6]:
key = []
value = []

for i,foldName in enumerate(files):
    key.append(foldName)
    value.append(i)
    print(i,foldName)
    
label_mapping = dict(zip(key,value))
label_value_key = dict(zip(value, key))

0 蘋果專區_網頁資料(10000).csv
1 AI與大數據_網頁資料(6482).csv
2 汽車_網頁資料(10005).csv
3 財經_網頁資料(10024).csv
4 旅遊_網頁資料(10021).csv
5 美妝保養_網頁資料(5272).csv
6 3C產品_網頁資料(2624).csv
7 醫療健康_網頁資料(10016).csv
8 職涯_網頁資料(10012).csv
9 美食_網頁資料(10000).csv
10 理財_網頁資料(7480).csv
11 無人駕駛_網頁資料(2003).csv
12 房地產_網頁資料(3951).csv
13 區塊鏈技術_網頁資料(4735).csv


In [7]:
file_path = []
for f in files:
    if f !='.ipynb_checkpoints':
        path = '/project/jt071-group23/news/'+ f 
        file_path.append(path)

In [70]:
data_content = []
data_label = []
all_data = pd.DataFrame()
#class
tag = 0

for path in file_path: 
    data = pd.read_csv(path)[['content','title']]
    data['class'] = tag
    
    # 刪除 重複的 新聞
    data = data.drop_duplicates('content', keep='first')
    
    #根據觀察 刪除字元<12的文章
    data = data[data['content'].apply(lambda x: len(str(x)) > 12)]
    
    # 刪除 無 內文 or url
    data = data.dropna()
    fileName = path.split('/')[-1]
    print("tag:",tag, fileName, '|', data.shape[0],)
    
    tag+=1   
    # 將每個類別中的content加到all_content
    all_data = all_data.append(data)
            
    # 取出每個content的類別
    # fileName = path.split('/')[-1]
    # label = label_mapping.get(fileName)
    # data_label.append(label)
        
print('all content',len(all_data))

tag: 0 蘋果專區_網頁資料(10000).csv | 9228
tag: 1 AI與大數據_網頁資料(6482).csv | 6319
tag: 2 汽車_網頁資料(10005).csv | 9474
tag: 3 財經_網頁資料(10024).csv | 9711
tag: 4 旅遊_網頁資料(10021).csv | 7593
tag: 5 美妝保養_網頁資料(5272).csv | 5021
tag: 6 3C產品_網頁資料(2624).csv | 2569
tag: 7 醫療健康_網頁資料(10016).csv | 9645
tag: 8 職涯_網頁資料(10012).csv | 9568
tag: 9 美食_網頁資料(10000).csv | 7706
tag: 10 理財_網頁資料(7480).csv | 7220
tag: 11 無人駕駛_網頁資料(2003).csv | 1930
tag: 12 房地產_網頁資料(3951).csv | 3890
tag: 13 區塊鏈技術_網頁資料(4735).csv | 4599
all content 94473


In [66]:
print(all_data.content.str.len().sort_values())

201        13
944        13
947        13
122        13
126        13
121        13
972        13
939        14
948        14
250        14
970        14
958        14
960        14
969        14
4727       15
893        15
1326       15
915        15
9447       15
955        15
919        16
2358       16
950        16
7152       16
125        16
1594       16
1545       16
937        17
1159       17
1951       17
        ...  
5686    20530
8529    20540
5141    20544
4092    20682
5738    20996
4719    21364
5982    21751
8764    21775
8563    21775
5192    21805
8613    21850
2812    22335
5254    22339
8045    22614
2930    23483
589     23650
8894    24875
6795    25492
5639    28384
4972    28818
5840    28982
5864    29384
8852    32289
6719    32289
9296    33098
4491    36798
840     39637
8951    40488
8944    43188
6129    47552
Name: content, Length: 94473, dtype: int64


In [75]:
all_data.shape

(94473, 3)

In [47]:
all_data.content_len[2]

2    10000
2     6482
2    10005
2    10024
2    10021
2     5272
2     2624
2    10016
2    10012
2    10000
2     7480
2     2003
2     3951
2     4735
Name: content_len, dtype: int64

接下來做重複部分處理

全留:all_data (94473, 3)

全刪:all_data_delete (69911, 3)  

隨機:all_data_sample (81790, 3)

In [71]:
#重複全殺(69911, 3)
all_data_delete= all_data.drop_duplicates('content', keep = False)

In [73]:
#隨機留一個(sample)
all_data_sample = all_data.groupby('content').apply(lambda x: x.sample(1)).reset_index(drop=True)

In [81]:
all_data.to_csv('../jt071-group23/jt071080/all_data.csv',index= False)

In [2]:
all_data= pd.read_csv('all_data.csv')

In [6]:
all_data.groupby('class')

In [4]:
all_data.concat(g for _, g in df.groupby("content") if len(g) > 1)

AttributeError: 'DataFrame' object has no attribute 'concat'

In [5]:
pandsaall_data

pandas.core.frame.DataFrame

In [6]:
test001=pd.date